# Mini-project: Prediction of Water Phases

The goal of this mini project is to compare the performance of different classifiers, in the task of predicting the phase of the water given its temperature and pressure. This project was done during the Automn Semester 2022 at the University of Fribourg.

---
## Libraries
---

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
import seaborn as sns
plt.rcParams['figure.figsize'] = (10, 7.5)
plt.rcParams['font.size'] = 15
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score as AUC
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier as KNN
import copy
import torch
from torch import nn, tensor
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
output_path = Path('output')
output_path.mkdir(exist_ok=True)

---
## Loading and handling the data
---

For the phase, the label 0, 1 and 2 correspond respectively to gas, liquid and solid. In this project, we will focus only on a temperature range $[200, 575]$ in Kelvin and a pressure range $[10^{-1},10^7]$ in Pascal.

In [ ]:
# Load the data and create a new column 'phase'
data = pd.read_csv("water_phases_sampling.csv")
label_to_phase = {0: "Gas", 1: "Liquid", 2: "Solid"}
data['phase'] = np.array([label_to_phase[label] for label in data['label']])
data

Filter the data to be sure it is in the given range

In [ ]:
# Filter the temperature and the pressure w.r.t. the given range
filter_temperature = data['temperature'].between(200, 575, inclusive="both")
data = data[filter_temperature]
filter_pressure = data['pressure'].between(1e-1, 1e7, inclusive="both")
data = data[filter_pressure]
data

### Proportions of the classes in the data

To have an idea of the proportions in the data

In [ ]:
def prop_classes(labels):
    unique, counts = np.unique(labels, return_counts=True)
    print(f"""This data has
{counts[0]/len(labels)} gas,
{counts[1]/len(labels)} liquid and
{counts[2]/len(labels)} solid samples (in proportions).""")

prop_classes(data['label'])

First issue here: class imbalance problem, we will need to reduce the number of samples of the *gas* class

### Plot the data

In [ ]:
sns.scatterplot(data=data, x='temperature', y='pressure', hue='phase', s=15)
None

This does not look good, try a logarithmic scale on the *pressure* axis

In [ ]:
plt.yscale('log')
sns.scatterplot(data=data, x='temperature', y='pressure', hue='phase', s=15)
plt.legend(loc='right')
None

Better. We will then need to preprocess the data of the *pressure* feature, by applying a *log10* transformation, so that we get a *linear scale* data on both temperature and pressure

### Preprocessing

In [ ]:
# Apply a log10 function to the pressure feature
data['pressure'] = np.log10(data['pressure'])
# Class imbalance problem, remove 80% of the gas samples and 10% of the liquid samples
balanced_data = data.drop(data[data['phase'] == 'Gas'].sample(frac=0.8).index)
balanced_data = balanced_data.drop(balanced_data[balanced_data['phase'] == 'Liquid'].sample(frac=0.1).index)
print(f"The balanced dataset has {len(balanced_data)} samples")
prop_classes(balanced_data['label'])

Data is now more balanced, it will be now split

### Train, validation and test split

In [ ]:
# Full data, used to plot
X = data[['temperature', 'pressure']]
y = data['label']
# Balanced data, will be splitted, used to train
X_balanced = balanced_data[['temperature', 'pressure']]
y_balanced = balanced_data['label']

# Split train and test (80-20%)
# random_state is used for reproducibility
X_fulltrain, X_test, y_fulltrain, y_test = train_test_split(X_balanced, y_balanced, test_size=0.2, random_state=17)
# Split the fulltrain (80%) to a sub-train and validation (0.25*0.8 = 20% of total)
X_train, X_val, y_train, y_val = train_test_split(X_fulltrain, y_fulltrain, test_size=0.25, random_state=73)

Plot the balanced data

In [ ]:
plt.yscale('log')
sns.scatterplot(x=balanced_data['temperature'], y=np.power(10, balanced_data['pressure']), hue=balanced_data['phase'], s=15)
plt.legend(loc='right')
None

Check if the propotions are roughly the same between the splits

In [ ]:
prop_classes(y_fulltrain)
prop_classes(y_test)
prop_classes(y_train)
prop_classes(y_val)

---
## Classifiers
---

In this part, three different classifiers will be compared. The pipeline is the same for all of them. In a first phase, using the train and validation sets, the hyperparameters are optimized in a grid search manner. After that, the train and validation sets are considered as the *fulltrain* set. The optimized model (i.e. with the best hyperparameters) is trained on it, and finally tested on the test set. The two metrics used are the accuracy and the area under the ROC curve (AUC), but the selection of the *best* model is chosen w.r.t. the accuracy first, then the AUC

In [ ]:
# Will held the results of each classifier on the test set
final_results = pd.DataFrame(columns=['classifier', 'accuracy', 'AUC'])

## Support vector machine

Hyperparameters optimization

In [ ]:
# Train and eval SKLearn model (here SVM and k-NN)
def train_eval_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = (predictions == y_test).sum() / len(predictions)
    proba_predictions = model.predict_proba(X_test)
    # AUC is computed in a 'one vs one' manner, averaged
    auc = AUC(y_test, proba_predictions, multi_class='ovo')
    return accuracy, auc

In [ ]:
# Define the hyperparameters for the grid search
kernels = ['rbf', 'poly']
c_values = np.logspace(-4, 10, 15)
gamma_values = np.logspace(-15, 1, 17)
degree_values = range(1, 4)
# Contains the hyperparameters and the result metrics
scores_svm = pd.DataFrame(columns=['kernel', 'C', 'gamma', 'degree', 'accuracy', 'auc'])

# Optimize the hyperparameters using grid search
for kernel in kernels:
    print(f"Optimizing {kernel} kernel")
    if kernel == 'rbf':
        for c in c_values:
            for gamma in gamma_values:
                # Set a max_iter to reduce the run time
                svm_model = SVC(C=c, gamma=gamma, kernel=kernel, probability=True, max_iter=100000, decision_function_shape='ovo')
                accuracy, auc = train_eval_model(svm_model, X_train, y_train, X_val, y_val)
                # Aggregate results
                new_row = pd.DataFrame({
                    'kernel': kernel,
                    'C': c,
                    'gamma': gamma,
                    'accuracy': np.around(accuracy, decimals=6),
                    'auc': np.around(auc, decimals=6),
                }, index=[0])
                scores_svm = pd.concat([scores_svm, new_row], ignore_index=True)
                # To show progression
                print('.', end='', flush=True)
                
    elif kernel == 'poly':
        for c in c_values:
            for gamma in gamma_values:
                for degree in degree_values:
                    svm_model = SVC(C=c, gamma=gamma, kernel=kernel, degree=degree, probability=True, max_iter=100000, decision_function_shape='ovo')
                    accuracy, auc = train_eval_model(svm_model, X_train, y_train, X_val, y_val)
                    new_row = pd.DataFrame({
                        'kernel': kernel,
                        'C': c,
                        'gamma': gamma,
                        'degree': degree,
                        'accuracy': np.around(accuracy, decimals=6),
                        'auc': np.around(auc, decimals=6),
                    }, index=[0])
                    scores_svm = pd.concat([scores_svm, new_row], ignore_index=True)
                    print('.', end='', flush=True)
    print()

# Sort the scores and print the best hyperparameters
sorted_scores_svm = scores_svm.sort_values(by=['accuracy', 'auc'], ascending=False)
print()
# Best model is the first in the sorted list
print(f"The best accuracy is with \n {sorted_scores_svm.iloc[0]}")
sorted_scores_svm

Take the best hyperparameters, and run the model with the *fulltrain* set as training set. Finally test it on the official test set

In [ ]:
# Remove columns of the metrics, and where there are NaN
best_hyperparams_svm = sorted_scores_svm.iloc[0].drop(['accuracy', 'auc']).dropna()
# Build the best model, train and evaluate it
best_model_svm = SVC(**best_hyperparams_svm, probability=True, decision_function_shape='ovo')
acc, auc = train_eval_model(best_model_svm, X_fulltrain, y_fulltrain, X_test, y_test)
# Aggregate results
new_row = pd.DataFrame({
    'classifier': 'SVM',
    'accuracy': acc,
    'AUC': auc,
}, index=[0])
final_results = pd.concat([final_results, new_row], ignore_index=True)
final_results

Predict results on the full original dataset and plot the results

In [ ]:
# Custom color map
colors = ["#3B4CC0", "#B40426", "#F6BE00"]
color_map = ListedColormap(colors)

# Creating meshgrid for the background plot
x_arr = np.linspace(200, 575, 300)
y_arr = np.linspace(-1, 7, 300)
xs, ys = np.meshgrid(x_arr, y_arr)
# Input data for predictions
input_data = np.c_[xs.ravel(), ys.ravel()]

In [ ]:
def plot_points_and_background(xs, ys, predictions, color_map, classifier):
    plt.yscale('log')
    # Transform pressure feature to initial numbers
    plt.pcolormesh(xs, np.power(10, ys), predictions, cmap=color_map, alpha=0.5)
    # Plot the data points and add title and labels
    plt.scatter(X['temperature'], np.power(10, X['pressure']), c=y, s=20, cmap=color_map, edgecolor='k')
    plt.xlabel("Temperature [K]")
    plt.ylabel("Pressure [Pa]")
    plt.title(f"Classification with {classifier}")
    # Handle legends
    patch0 = mpatches.Patch(color=colors[0], label='Gas')
    patch1 = mpatches.Patch(color=colors[1], label='Liquid')
    patch2 = mpatches.Patch(color=colors[2], label='Solid')
    patch3 = mpatches.Patch(color='k', label='Transition line')
    plt.legend(handles=[patch0, patch1, patch2, patch3], loc="lower right")

    # Plot theoretical transition lines
    # Equations are taken from the paper "New Equations for the Sublimation Pressure and Melting Pressure of H2O Ice Ih"
    # Triple point pressure and temperature of water
    pt = 611.657
    tt = 273.16
    # Sublimation line
    sublimation_fn = lambda T: pt * np.exp((-0.212144006e2 * (T/tt)**0.00333333333
              + 0.273203819e2 * (T/tt)**1.20666667
              - 0.610598130e1 * (T/tt)**1.70333333) * tt / T)
    # Vaporization line
    vaporization_fn = lambda T: pt * np.exp(-(42e3/8.3145) * (1/T - 1/tt))
    # Compute values of pressure
    inf_points = xs[xs <= tt]
    sup_points = xs[xs > tt]
    sublimation_vals = sublimation_fn(inf_points)
    vaporization_vals = vaporization_fn(sup_points)
    # Plot the three transition lines
    sns.lineplot(x=inf_points, y=sublimation_vals, c='k', linewidth=2)
    sns.lineplot(x=[tt, 272.74247492], y=[pt, 1e7], c='k', linewidth=2)
    sns.lineplot(x=sup_points, y=vaporization_vals, c='k', linewidth=2)

    # Export the figure
    plt.savefig(output_path / f"classifier_{classifier}.pdf")
    None

In [ ]:
# Predictions for the whole dataset
predictions = best_model_svm.predict(input_data)
predictions = predictions.reshape(xs.shape)
plot_points_and_background(xs, ys, predictions, color_map, 'SVM')

## $k$-nearest neighbors

Hyperparameters optimization

In [ ]:
# Define the hyperparameters for the grid search
n_neighbors = range(1, 8)
weights = ['uniform', 'distance']
algorithms = ['ball_tree', 'kd_tree', 'brute']
leaf_sizes = range(1, 51)
p_values = [1, 2, 3]
# Contains the hyperparameter and the result metrics
scores_knn = pd.DataFrame(columns=['n_neighbors', 'weights', 'algorithm', 'leaf_size', 'p', 'accuracy', 'auc'])

# Optimize the hyperparameters using grid search
for algorithm in algorithms:
    print(f"Optimizing {algorithm}")
    for weight in weights:
        for n in n_neighbors:
            for p_value in p_values:
                if algorithm == 'brute':
                    # Create, train and evaluate the model
                    knn_model = KNN(n, weights=weight, algorithm=algorithm, p=p_value)
                    accuracy, auc = train_eval_model(knn_model, X_train, y_train, X_val, y_val)
                    # Aggregate results
                    new_row = pd.DataFrame({
                        'n_neighbors': n,
                        'weights': weight,
                        'algorithm': algorithm,
                        'p': p_value,
                        'accuracy': np.around(accuracy, decimals=6),
                        'auc': np.around(auc, decimals=6),
                    }, index=[0])
                    scores_knn = pd.concat([scores_knn, new_row], ignore_index=True)
                    print('.', end='', flush=True)
                    
                else:
                    for leaf_size in leaf_sizes:
                        knn_model = KNN(n, weights=weight, algorithm=algorithm, leaf_size=leaf_size, p=p_value)
                        accuracy, auc = train_eval_model(knn_model, X_train, y_train, X_val, y_val)
                        new_row = pd.DataFrame({
                            'n_neighbors': n,
                            'weights': weight,
                            'algorithm': algorithm,
                            'leaf_size': leaf_size,
                            'p': p_value,
                            'accuracy': np.around(accuracy, decimals=6),
                            'auc': np.around(auc, decimals=6),
                        }, index=[0])
                        scores_knn = pd.concat([scores_knn, new_row], ignore_index=True)
                        print('.', end='', flush=True)  
    print()
    
# Sort the scores and print the best hyperparameters
sorted_scores_knn = scores_knn.sort_values(by=['accuracy', 'auc'], ascending=False)
print()
# Best model is the first in the sorted list
print(f"The best accuracy is with \n {sorted_scores_knn.iloc[0]}")
sorted_scores_knn

Take the best hyperparameters, and run the model with the *fulltrain* set as training set. Finally test it on the official test set

In [ ]:
# Remove columns of the metrics, and where there are NaN
best_hyperparams_knn = sorted_scores_knn.iloc[0].drop(['accuracy', 'auc']).dropna()
# Build, train and evaluate the best model
best_model_knn = KNN(**best_hyperparams_knn)
acc, auc = train_eval_model(best_model_knn, X_fulltrain, y_fulltrain, X_test, y_test)
# Aggregate results
new_row = pd.DataFrame({
    'classifier': 'k-NN',
    'accuracy': acc,
    'AUC': auc,
}, index=[0])
final_results = pd.concat([final_results, new_row], ignore_index=True)
final_results

Predict results on the full original dataset and plot the results

In [ ]:
# Predictions for the whole dataset
predictions = best_model_knn.predict(input_data)
predictions = predictions.reshape(xs.shape)
plot_points_and_background(xs, ys, predictions, color_map, 'k-NN')

## Neural network (multilayer perceptron)

Handling the data. The data and labels will be stored in a PyTorch dataset. This dataset will then be wrapped in a dataloader, that gives batches when training

In [ ]:
# A Dataset subclass needs to implement __len__() and __getitem__()
class WaterDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # dtype is float32 so that it will be the same data type as the weights in the MLP
        item_features = tensor(self.features.iloc[idx], dtype=torch.float32)
        # dtype long is int64
        item_label = tensor(self.labels.iloc[idx], dtype=torch.long)
        return item_features, item_label
    
# Create all three datasets
train_set = WaterDataset(X_train, y_train)
val_set = WaterDataset(X_val, y_val)
test_set = WaterDataset(X_test, y_test)

Create the multilayer perceptron

In [ ]:
class MLP(nn.Module):
    def __init__(self, n_hidden_layers=2, n_units=20):
        super(MLP, self).__init__()
        self.n_hidden_layers = n_hidden_layers
        self.n_units = n_units
        assert self.n_hidden_layers >= 0
        assert self.n_units >= 1
        # If a GPU is available, use it for training
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        
        # Input has size 2 (temperature and pressure)
        self.first_layer = nn.Linear(2, n_units)
        # Create the number of hidden layers we want, with the number of units (neurons) we specified
        self.hidden_layers = nn.ModuleList([nn.Linear(n_units, n_units) for i in range(n_hidden_layers)])
        # Output has size 3 (gas, liquid or solid)
        self.output_layer = nn.Linear(n_units, 3)
        # Activation function between layers
        self.act_fn = nn.Sigmoid()
        # Softmax so that we get probabilities for the 3 classes
        self.softmax = nn.Softmax(dim=1)
        
    # Forward pass
    def forward(self, x):
        # Input layer and activation
        x = self.act_fn(self.first_layer(x))
        # Hidden layers and activation
        if self.n_hidden_layers != 0:
            for layer in self.hidden_layers:
                x = self.act_fn(layer(x))
        # Output layer
        x = self.output_layer(x)
        return x

Main train method for the MLP models

In [ ]:
def train_mlp(model, optimizer, epochs, batch_size, learning_rate, train_dataset, validation_dataset, test_dataset):
    # Use GPU if available
    model.to(model.device)
    # Set model to training mode
    model.train()
    # Create the dataloader and the optimizer
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    optimizer = optimizer(model.parameters(), lr=learning_rate)
    loss_fn = nn.CrossEntropyLoss()
    # To store the metrics
    metrics = {
        'train': pd.DataFrame(columns=['loss', 'acc', 'auc']),
        'validation': pd.DataFrame(columns=['loss', 'acc', 'auc']),
        'test': pd.DataFrame(columns=['loss', 'acc', 'auc']),
    }
    # To save the best model (early stopping)
    best_acc = 0
    
    # Training loop
    for epoch in tqdm(range(1, epochs+1)):
        for features, labels in train_dataloader:
            # Send to same device as the model
            features = features.to(model.device)
            labels = labels.to(model.device)
            outputs = model(features)
            # Reset gradients to 0
            optimizer.zero_grad()
            loss = loss_fn(outputs, labels)
            # Compute gradients (backpropagation)
            loss.backward()
            # Update weights
            optimizer.step()
            
        # Evaluate the model on the train set
        loss, acc, auc = eval_mlp(model, train_dataset, loss_fn)
        # Aggregate the metrics
        new_row = pd.DataFrame({
            # Item returns the single element in the loss tensor
            'loss': loss.item(),
            'acc': acc,
            'auc': auc,
        }, index=[0])
        metrics['train'] = pd.concat([metrics['train'], new_row], ignore_index=True)
        
        # Evaluate the model on the validation set
        loss, acc, auc = eval_mlp(model, validation_dataset, loss_fn)
        # Aggregate the metrics
        new_row = pd.DataFrame({
            'loss': loss.item(),
            'acc': acc,
            'auc': auc,
        }, index=[0])
        metrics['validation'] = pd.concat([metrics['validation'], new_row], ignore_index=True)
        
        if acc > best_acc:
            best_acc = acc
            # Create a new reference in memory
            model_to_save = copy.deepcopy(model)
            torch.save(model_to_save.state_dict(), 'best_model.pth')
            
    # Load the best model and evaluate it on the test set
    best_model = copy.deepcopy(model)
    best_model.load_state_dict(torch.load('best_model.pth'))
    loss, acc, auc = eval_mlp(best_model, test_dataset, loss_fn)
    new_row = pd.DataFrame({
        'loss': loss.item(),
        'acc': acc,
        'auc': auc,
    }, index=[0])
    metrics['test'] = pd.concat([metrics['test'], new_row], ignore_index=True)
        
    return metrics, best_model

Evaluation method for the MLP models

In [ ]:
# Disable gradient calculation            
@torch.no_grad()            
def eval_mlp(model, dataset, loss_fn):
    model.to(model.device)
    # Set model to eval mode
    model.eval()
    dataloader = DataLoader(dataset, 64, shuffle=True)
    # Store all labels and predictions to compute metrics all at the same time
    full_labels = torch.tensor([]).to(model.device)
    full_predictions = torch.tensor([]).to(model.device)
    # To compute the prediction
    for features, labels in dataloader:
        # Send to same device as the model
        features = features.to(model.device)
        labels = labels.to(model.device)
        predictions = model(features)
        loss = loss_fn(predictions, labels)
        # Concatenate labels and predictions of this batch to the other ones
        full_labels = torch.cat((full_labels.long(), labels), 0)
        full_predictions = torch.cat((full_predictions, predictions), 0)

    # Compute the metrics (loss, accuracy and AUC)
    loss = loss_fn(full_predictions, full_labels)
    correct_pred = (full_predictions.argmax(1) == full_labels).type(torch.float).sum().item()
    accuracy = correct_pred / len(dataset)
    full_predictions = model.softmax(full_predictions)
    # cpu() sends the tensor to the CPU
    auc = AUC(full_labels.cpu(), full_predictions.cpu(), multi_class='ovo')
    return loss, accuracy, auc

Bootstrap method to initialize the model. It creates models and evaluates them directly on the validation set. Take the best model as initialization for the training

In [ ]:
def bootstrap(n_models, n_layers, n_units, dataset, loss_fn, verbose=False):
    assert n_models >= 0
    # Base model
    best_model = MLP(n_layers, n_units)
    _loss, acc, auc = eval_mlp(best_model, dataset, loss_fn)
    best_acc = acc
    best_auc = auc
    
    # Create new models and compare the accuracy
    for i in range(n_models):
        model = MLP(n_layers, n_units)
        _loss, acc, auc = eval_mlp(model, dataset, loss_fn)
        if acc > best_acc:
            best_acc = acc
            best_auc = auc
            best_model = copy.deepcopy(model)
        if verbose: print('.', end='', flush=True)
                
    return acc, auc, best_model

Hyperparameters optimization: first try different number of layers and units, and get the best architecture using the bootstrap method. Using this architecture, then optimize the batch size and the learning rate

In [ ]:
n_layers_list = range(5)
n_units_list = range(10, 211, 20)
batch_sizes = [4, 8, 16, 32]
learning_rates = np.logspace(-6, 0, 7)
scores_architecture_mlp = pd.DataFrame(columns=['n_layers', 'n_units', 'accuracy', 'auc'])
# Contains the hyperparameter and the result metrics
scores_mlp = pd.DataFrame(columns=['batch_size', 'learning_rate', 'accuracy', 'auc'])

print()
print("Optimizing architecture")
# Get the most promising number of layers and neurons
for n_layers in n_layers_list:
    for n_units in n_units_list:
        acc, auc, start_model = bootstrap(500, n_layers, n_units, val_set, nn.CrossEntropyLoss())
        new_row = pd.DataFrame({
            'n_layers': n_layers,
            'n_units': n_units,
            'accuracy': acc,
            'auc': auc,
        }, index=[0])
        scores_architecture_mlp = pd.concat([scores_architecture_mlp, new_row], ignore_index=True)
        print('.', end='', flush=True)  
        
sorted_scores_architecture_mlp = scores_architecture_mlp.sort_values(by=['accuracy', 'auc'], ascending=False)
# Best model is the first in the sorted list
print()
print(f"The best accuracy is with \n {sorted_scores_architecture_mlp.iloc[0]}")
best_n_layers, best_n_units, _, _ = sorted_scores_architecture_mlp.iloc[0]
print(sorted_scores_architecture_mlp)
     
print()    
print("Optimizing hyperparameters")
# Initialze a (hopefully) promising model
_, _, start_model = bootstrap(5000, best_n_layers, best_n_units, val_set, nn.CrossEntropyLoss(), verbose=True)
for batch_size in batch_sizes:
    for learning_rate in learning_rates:
        # Define all parameters we need for the training
        kwargs = {
            'model': start_model,
            'optimizer': torch.optim.Adam,
            'epochs': 100,
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'train_dataset': train_set,
            'validation_dataset': val_set,
            # Set the test as the validation set
            'test_dataset': val_set,
        }
        metrics, _trained_model = train_mlp(**kwargs)
        # Aggregate stats
        new_row = pd.DataFrame({
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'accuracy': np.around(metrics['test']['acc'], decimals=6),
            'auc': np.around(metrics['test']['auc'], decimals=6),
        }, index=[0])
        scores_mlp = pd.concat([scores_mlp, new_row], ignore_index=True)
                
# Sort the scores and print the best hyperparameters
sorted_scores_mlp = scores_mlp.sort_values(by=['accuracy', 'auc'], ascending=False)
# Best model is the first in the sorted list
print()
print(f"The best accuracy is with \n {sorted_scores_mlp.iloc[0]}")
sorted_scores_mlp

Take the best hyperparameters, and train the model. Finally test it on the test set

In [ ]:
print("Bootstrap model")
_, _, start_model = bootstrap(5000, best_n_layers, best_n_units, val_set, nn.CrossEntropyLoss(), verbose=True)
# Take the best hyperparameters
best_batch_size, best_learning_rate, _, _ = sorted_scores_mlp.iloc[0]
kwargs = {
    'model': start_model,
    'optimizer': torch.optim.Adam,
    'epochs': 500,
    'batch_size': best_batch_size,
    'learning_rate': best_learning_rate,
    'train_dataset': train_set,
    'validation_dataset': val_set,
    'test_dataset': test_set,
}
metrics, best_model_mlp = train_mlp(**kwargs)

# Aggregate the results of the test set
acc = metrics['test']['acc']
auc = metrics['test']['auc']
new_row = pd.DataFrame({
    'classifier': 'MLP',
    'accuracy': acc,
    'AUC': auc,
}, index=[0])
final_results = pd.concat([final_results, new_row], ignore_index=True)
final_results

Predict results on the full original dataset and plot the results

In [ ]:
# Predictions for the full dataset
input_data = torch.tensor(input_data, dtype=torch.float32).to(best_model_mlp.device)
raw_predictions = best_model_mlp(input_data)
predictions = best_model_mlp.softmax(raw_predictions).argmax(1)
predictions = predictions.cpu().numpy().reshape(xs.shape)
plot_points_and_background(xs, ys, predictions, color_map, 'MLP')